In [45]:
import requests
import json
import xmltodict
import urllib.parse
import opencitingpy
import datetime
import sqlite3
import os

# Setup sqlite Datenbank
Datenbank wird im Verzeichnis "C:\MA_Pethke_3992454" abgelegt

In [48]:
# Anlegen von Datenbankspeicherort, falls er nicht existiert
path = "C:\MA_Pethke_3992454"
if not os.path.exists(path):
    os.makedirs(path)

# Verbindung zu DB aufbauen, DB wird automatisch erstellt, wenn keine da ist
con = sqlite3.connect("C:\MA_Pethke_3992454\MA_3992454.db")
cur = con.cursor()

In [119]:
## Tabellen Schema erstellen
#------------------- Tabelle Dokument --------------------------------------#
cur.execute("CREATE TABLE IF NOT EXISTS Dokument(PMID INTEGER, titel TEXT, pubdate DATE)")

#------------------- Tabelle Zitationen --------------------------------------#
cur.execute("CREATE TABLE IF NOT EXISTS Zitationen(PMID INTEGER, jahr YEAR, count INTEGER)")

#------------------- Tabelle Mesh --------------------------------------#
cur.execute("CREATE TABLE IF NOT EXISTS Mesh(UI VARCHAR, name TEXT, datum DATE)")

#------------------- Tabelle Abfrage --------------------------------------#
cur.execute("CREATE TABLE IF NOT EXISTS Abfrage(query TEXT, PMID INTEGER, ranking INTEGER, mesh VARCHAR)")
con.commit()

# PubMed
## Teil 1
### Query mit Stichworten --> Liste der relevanten Dokumente
queryPubMed: Stichwortartige Suche <br>
idList: Liste mit PMIDs <br>
countResult: Anzahl (maximal) zurückgegebener Dokumente <br>

### URL 
db: welche Datenbank wird durchsucht (pubmed) <br>
term: queryPubMed <br>
RetMax: countResult <br>
sort: Art der Sortierung (default: relevance) <br>
retmod: Format der zurückgelieferten Daten <br>



In [113]:
def logQuery(query):
    now = datetime.datetime.now()
    idq = nextIteration("Q")
    cur.execute("INSERT INTO Q (IDQ, query, timestamp) VALUES ('"+str(idq)+"', '"+query+"', '"+str(now)+"')")
    con.commit()
    return idq

In [114]:
## Liste der "relevanten" Dokumente
queryPubMed = "head injury"
countResult = 10
responsePubMedID = requests.get('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term='+urllib.parse.quote(queryPubMed)+'&RetMax='+urllib.parse.quote(str(countResult))+'&sort=relevance&retmode=json')
responsePubMedID_json = json.loads(responsePubMedID.text)
idList = responsePubMedID_json["esearchresult"]["idlist"]
print(idList)

['25335757', '36637470', '22942365', '9279162', '9425562', '9339463', '3330962', '3931543', '27509653', '23431655']


## Teil 2
### Query mit PMID aus Teil 1 --> Liste der MeSH zu dem jeweiligen Dokument
Für jedes Dokument (Identifier: PMID) wird eine Detailsuche durchgeführt. <br>
Extrahierte Infos: <br>
Titel <br>
Liste der MeSH: [[PMID, [MeSH UI, MeSH Name], [MeSH UI, MeSH Name]] ,[PMID, [MeSH UI, MeSH Name], [MeSH UI, MeSH Name]] ,  ...] (beliebig viele MeSH pro Dokument)<br>
Datum Veröffentlichung: Wann wurde das Dokument veröffentlicht <br>

In [118]:
## Liste der MeSHs (zu allen Dokumenten)
idp = 0
for pmid in idList:
    idp = idp + 1
    # print(id)
    # temporäre Liste zum Anhängen an meshList
    # Abfrage zu einer einzelnen PMID
    response_PubMedMesh = requests.get("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id="+urllib.parse.quote(str(pmid)))
    # Formatierung
    xpars = xmltodict.parse(response_PubMedMesh.text)
    json_str = json.dumps(xpars, indent=4)
    json_data = json.loads(json_str)
    title = json_data["PubmedArticleSet"]["PubmedArticle"]["MedlineCitation"]["Article"]["ArticleTitle"]
    # Nicht alle Dokumente haben ein eigenes Veröffentlichungsdatum, dann wird sich auf das DateCompleted von PubMed berufen (TODO: recherchieren, was das genau ist)
    try:
        pubDateAll = json_data["PubmedArticleSet"]["PubmedArticle"]["MedlineCitation"]["Article"]["ArticleDate"]
        pubDate = datetime.datetime(int(pubDateAll["Year"]), int(pubDateAll["Month"]), int(pubDateAll["Day"]))
    except:
        pubDateAll = json_data["PubmedArticleSet"]["PubmedArticle"]["MedlineCitation"]["DateCompleted"]
        pubDate = datetime.datetime(int(pubDateAll["Year"]), int(pubDateAll["Month"]), int(pubDateAll["Day"]))
    meshAll = json_data["PubmedArticleSet"]["PubmedArticle"]["MedlineCitation"]["MeshHeadingList"]["MeshHeading"]
    for mesh in meshAll:
        meshUI = mesh["DescriptorName"]["@UI"]
        cur.execute("SELECT * FROM M WHERE meshui = '"+str(meshUI)+"'")
        if cur.fetchall() == []:
            meshName = mesh["DescriptorName"]["#text"]
            cur.execute("INSERT INTO M (meshui, meshname) VALUES ('"+str(meshUI)+"', '"+str(meshName)+"')")
        cur.execute("INSERT INTO P (IDP, IDQ, PMID, pubdate, meshui) VALUES ('"+str(idp)+"', '"+str(idqAkt)+"', '"+str(pmid)+"', '"+str(pubDate)+"', '"+str(meshUI)+"')")
        con.commit()
        

In [ ]:
def fillTableP():
    cur.execute("INSERT INTO P (IDP, IDQ, PMID, pubdate, meshui) VALUES ('"+str(idp)+"', '"+str(idqAkt)+"', '"+str(pmid)+"', '"+str(pubDate)+"', '"+str(meshUI)+"')")
    cur.execute("INSERT INTO M (IDM, meshui, meshname) VALUES ('"+str(highestM+1)+"', '"+str(meshUI)+"', '"+str(meshName)+"')")
    con.commit()

In [97]:
print(meshUI)

D012887


In [100]:
cur.execute("SELECT * FROM M WHERE meshui = '"+str(meshUI)+"'")

In [102]:
print(cur.fetchall())

[]


In [5]:
print()

[['25335757', ['D000293', 'Adolescent'], ['D000328', 'Adult'], ['D002648', 'Child'], ['D006259', 'Craniocerebral Trauma'], ['D006801', 'Humans'], ['D017410', 'Practice Guidelines as Topic'], ['D012307', 'Risk Factors'], ['D013124', 'Spinal Injuries'], ['D014057', 'Tomography, X-Ray Computed'], ['D014218', 'Triage']], ['36637470', ['D007223', 'Infant'], ['D002648', 'Child'], ['D006801', 'Humans'], ['D002675', 'Child, Preschool'], ['D002649', 'Child Abuse'], ['D006259', 'Craniocerebral Trauma'], ['D012640', 'Seizures'], ['D059906', 'Neuroimaging']], ['22942365', ['D061906', 'Advanced Trauma Life Support Care'], ['D058109', 'Airway Management'], ['D002648', 'Child'], ['D006259', 'Craniocerebral Trauma'], ['D004677', 'Encephalocele'], ['D015600', 'Glasgow Coma Scale'], ['D006801', 'Humans'], ['D010102', 'Oxygen Inhalation Therapy'], ['D014057', 'Tomography, X-Ray Computed']], ['9279162', ['D000059', 'Accidents'], ['D000293', 'Adolescent'], ['D002648', 'Child'], ['D002675', 'Child, Preschoo

# MeSH
## Wann wurde das MeSH hinzugefügt?
Dies ist für jedes MeSH in allen Dokumenten zu ermitteln

Liste der MeSH: [[PMID, [MeSH UI, MeSH Name], [MeSH UI, MeSH Name]] ,[PMID, [MeSH UI, MeSH Name], [MeSH UI, MeSH Name]] ,  ...] (beliebig viele MeSH pro Dokument)<br>
Diese Liste wird um das Datum ergänzt: [[PMID, [MeSH UI, MeSH Name, MeSH Datum], [MeSH UI, MeSH Name, MeSH Datum]] ,[PMID, [MeSH UI, MeSH Name, MeSH Datum], [MeSH UI, MeSH Name, MeSH Datum]] ,  ...]

In [6]:
for dok in range(len(meshList)):
    # print(dok[0])
    i = 0
    for mesh in range(len(meshList[dok])):
        if i != 0:
            response = requests.get('https://id.nlm.nih.gov/mesh/'+str(meshList[dok][mesh][0])+'.json', headers={'Accept': 'application/json'})
            # data = response.json()
            # meshList[dok][mesh].append(datetime.datetime.strptime(data["dateCreated"], '%Y-%m-%d'))
        i = i+1

In [7]:
print(meshList[0])

['25335757', ['D000293', 'Adolescent'], ['D000328', 'Adult'], ['D002648', 'Child'], ['D006259', 'Craniocerebral Trauma'], ['D006801', 'Humans'], ['D017410', 'Practice Guidelines as Topic'], ['D012307', 'Risk Factors'], ['D013124', 'Spinal Injuries'], ['D014057', 'Tomography, X-Ray Computed'], ['D014218', 'Triage']]


In [ ]:
## semantic scholar

In [39]:
def getCitations(offset, pmid):
    citationsPerYear = {}
    response = requests.get("https://api.semanticscholar.org/graph/v1/paper/PMID:"+str(pmid)+"/citations?fields=year&offset="+str(offset)+"&limit="+str(1000))
    json_str = json.loads(response.text)
    citations = json_str["data"]
    citationCount = len(citations)
    for cite in citations:
        year = str(cite["citingPaper"]["year"])
        if year not in citationsPerYear:
            citationsPerYear[year] = 1
        else:
            citationsPerYear[year] = citationsPerYear[year] + 1
    return citationsPerYear, citationCount

In [ ]:
# def addCitesToPMID(citationsPerYear):
#     for year in citationsPerYear:

In [9]:
offset = 0

for id in idList:
    citeCount = getCitations(offset, id)
    while citeCount[1] != 0:
        

{'offset': 0, 'data': [{'citingPaper': {'paperId': 'b6de1e809ad74fc2b438536c84a1c289006ad4db', 'year': 2023}}, {'citingPaper': {'paperId': '7a6754077f31a797d593bc37c9fe08ac7ba494e8', 'year': 2022}}, {'citingPaper': {'paperId': '8877efd382fda0823ec3dac72eb551ca8f91faf8', 'year': 2022}}, {'citingPaper': {'paperId': 'c67e1bc96798dae648950ee89bf82e3e01c63728', 'year': 2022}}, {'citingPaper': {'paperId': '008453b90cac84e3b0ae68d46cdf13470c9e8f19', 'year': 2022}}, {'citingPaper': {'paperId': 'efa0a82ec2bec4bc25c9e15296bebb68b7113b1c', 'year': 2022}}, {'citingPaper': {'paperId': '0f2bf3d8ec81ca36527d32d2d9a8db59949656c4', 'year': 2022}}, {'citingPaper': {'paperId': 'ef551092b64b0ed5914f673738b52907fe91acad', 'year': 2022}}, {'citingPaper': {'paperId': '72f11e8afe5b22c715617feb33c5f97ab40a0666', 'year': 2022}}, {'citingPaper': {'paperId': 'a18913ab3a1e112cc4ea16d9498902b916cf3fbf', 'year': 2022}}, {'citingPaper': {'paperId': '3c561f4ecbf92d12f434d3dd443e9158e61938fb', 'year': 2022}}, {'citingP

In [13]:
offset = 0
limit = 1000
pmid = 25335757
response = requests.get("https://api.semanticscholar.org/graph/v1/paper/PMID:"+str(pmid)+"/citations?fields=year&offset="+str(offset)+"&limit="+str(limit))
json_str = json.loads(response.text)
citationCount = len(json_str["data"])
print(json_str["data"])
while citationCount != 0:
    print(json_str)
    offset = offset + limit
    response = requests.get("https://api.semanticscholar.org/graph/v1/paper/PMID:"+str(pmid)+"/citations?fields=year&offset="+str(offset)+"&limit="+str(limit))
    json_str = json.loads(response.text)
    citationCount = len(json_str["data"])
    # print(citationCount)
# print(json_str["data"])
# print(len(json_str["data"]))


[{'citingPaper': {'paperId': 'b6de1e809ad74fc2b438536c84a1c289006ad4db', 'year': 2023}}, {'citingPaper': {'paperId': '7a6754077f31a797d593bc37c9fe08ac7ba494e8', 'year': 2022}}, {'citingPaper': {'paperId': '8877efd382fda0823ec3dac72eb551ca8f91faf8', 'year': 2022}}, {'citingPaper': {'paperId': 'c67e1bc96798dae648950ee89bf82e3e01c63728', 'year': 2022}}, {'citingPaper': {'paperId': '008453b90cac84e3b0ae68d46cdf13470c9e8f19', 'year': 2022}}, {'citingPaper': {'paperId': 'efa0a82ec2bec4bc25c9e15296bebb68b7113b1c', 'year': 2022}}, {'citingPaper': {'paperId': '0f2bf3d8ec81ca36527d32d2d9a8db59949656c4', 'year': 2022}}, {'citingPaper': {'paperId': 'ef551092b64b0ed5914f673738b52907fe91acad', 'year': 2022}}, {'citingPaper': {'paperId': '72f11e8afe5b22c715617feb33c5f97ab40a0666', 'year': 2022}}, {'citingPaper': {'paperId': 'a18913ab3a1e112cc4ea16d9498902b916cf3fbf', 'year': 2022}}, {'citingPaper': {'paperId': '3c561f4ecbf92d12f434d3dd443e9158e61938fb', 'year': 2022}}, {'citingPaper': {'paperId': 'ef

In [42]:
getCitations(0, 25335757)[1]

120